# 🏥 Enhanced Medical NER Training - Production-Ready
## Advanced Named Entity Recognition for French Medical Documents

**Objectives:**
- 🎯 Train a high-performance medical NER model
- 🇫🇷 Optimized for French medical terminology
- 📊 100+ diverse training samples across 12 entity types
- 🚀 CamemBERT-based (French-optimized BERT)
- 📈 Advanced evaluation with per-entity metrics
- 💾 Production deployment package

**Entity Types:**
`DISEASE` | `MEDICATION` | `SYMPTOM` | `DOSAGE` | `DATE` | `PROCEDURE` | `ANATOMY` | `TEST` | `LAB_VALUE` | `AGE` | `GENDER` | `FREQUENCY`

---
**⚙️ Recommended Colab Settings:**
- Runtime → Change runtime type → GPU (Tesla T4)
- Edit → Notebook settings → GPU hardware accelerator

**📦 Model:** CamemBERT-ner (French medical BERT)  
**⏱️ Training Time:** ~15-20 minutes on Tesla T4  
**🎓 Perfect for:** Academic projects, medical document processing, research

## 🔧 Step 1: Environment Setup & GPU Verification

In [ ]:
import torch
import sys
from datetime import datetime

print("="*70)
print("🔍 ENVIRONMENT VERIFICATION")
print("="*70)

# Python version
print(f"\n🐍 Python Version: {sys.version.split()[0]}")

# PyTorch version
print(f"🔥 PyTorch Version: {torch.__version__}")

# GPU detection
if torch.cuda.is_available():
    print(f"\n✅ GPU AVAILABLE!")
    print(f"   ├─ Device: {torch.cuda.get_device_name(0)}")
    print(f"   ├─ Compute Capability: {torch.cuda.get_device_capability(0)}")
    print(f"   ├─ Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   └─ CUDA Version: {torch.version.cuda}")
    device = torch.device('cuda')
    
    # Memory check
    torch.cuda.empty_cache()
    print(f"\n💾 GPU Memory Status:")
    print(f"   ├─ Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"   └─ Reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
else:
    print(f"\n⚠️  NO GPU DETECTED!")
    print(f"   └─ Training will be extremely slow. Enable GPU in Runtime settings!")
    device = torch.device('cpu')

print(f"\n🎯 Selected Device: {device}")
print(f"⏰ Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

## 📦 Step 2: Install Dependencies

Installing state-of-the-art NLP libraries optimized for token classification.

In [ ]:
%%capture install_output
!pip install -q transformers==4.37.0 datasets==2.16.1 accelerate==0.26.1 seqeval==1.2.2 scikit-learn matplotlib

# Show installation summary
print("✅ Installation Complete!")
print("\n📦 Installed Packages:")
print("   ├─ transformers: 4.37.0 (Hugging Face)")
print("   ├─ datasets: 2.16.1 (Data processing)")
print("   ├─ accelerate: 0.26.1 (Training optimization)")
print("   ├─ seqeval: 1.2.2 (NER metrics)")
print("   ├─ scikit-learn (ML utilities)")
print("   └─ matplotlib (Visualization)")

## 🔄 Step 3: Import Libraries & Set Random Seeds

In [ ]:
# Core libraries
import torch
import torch.nn as nn
import numpy as np
import json
import random
import os
import shutil
from collections import Counter
from typing import Dict, List, Tuple

# Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
    EarlyStoppingCallback
)

# Data processing
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Metrics
from seqeval.metrics import (
    classification_report as seqeval_report,
    f1_score,
    precision_score,
    recall_score
)

# Visualization
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
def set_seed(seed=42):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)
print("✅ Libraries imported successfully!")
print("✅ Random seeds set for reproducibility (seed=42)")

## 🏷️ Step 4: Define Enhanced Entity Labels (12 Types)

Extended label set for comprehensive medical entity extraction.

In [ ]:
# BIO tagging format (Beginning, Inside, Outside)
# 12 entity types for comprehensive medical NER
labels = [
    'O',  # Outside any entity
    'B-DISEASE', 'I-DISEASE',           # Diseases, conditions
    'B-MEDICATION', 'I-MEDICATION',     # Drugs, medicines
    'B-SYMPTOM', 'I-SYMPTOM',           # Symptoms, signs
    'B-DOSAGE', 'I-DOSAGE',             # Medication dosages, frequencies
    'B-DATE', 'I-DATE',                 # Dates, temporal expressions
    'B-PROCEDURE', 'I-PROCEDURE',       # Medical procedures, surgeries
    'B-ANATOMY', 'I-ANATOMY',           # Body parts, organs
    'B-TEST', 'I-TEST',                 # Lab tests, imaging
    'B-LAB_VALUE', 'I-LAB_VALUE',       # Lab result values
    'B-AGE', 'I-AGE',                   # Patient age
    'B-GENDER', 'I-GENDER',             # Patient gender
    'B-FREQUENCY', 'I-FREQUENCY'        # Medication frequency
]

label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}

# Entity categories for reporting
entity_types = ['DISEASE', 'MEDICATION', 'SYMPTOM', 'DOSAGE', 'DATE', 
                'PROCEDURE', 'ANATOMY', 'TEST', 'LAB_VALUE', 'AGE', 
                'GENDER', 'FREQUENCY']

print("="*70)
print("🏷️  ENHANCED LABEL SCHEMA")
print("="*70)
print(f"\n📊 Total labels: {len(labels)} (BIO format)")
print(f"📊 Entity types: {len(entity_types)}")
print(f"\n🎯 Entity Categories:")
for i, entity in enumerate(entity_types, 1):
    print(f"   {i:2d}. {entity}")

print(f"\n✅ Label mappings created!")
print(f"   ├─ label2id: {len(label2id)} mappings")
print(f"   └─ id2label: {len(id2label)} mappings")

## 📝 Step 5: Create Comprehensive Training Dataset (100+ Samples)

High-quality annotated French medical texts with diverse clinical scenarios.